In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import ipyplot
import torch
import pytorch_lightning as pl
from sklearn.metrics import f1_score
%matplotlib inline

In [4]:
from src.model import ImageClassifier
from src.utils import ImagesDataset, evaluate_model
from src.transforms import transform
from src import params

# Load dataset

In [5]:
labelled_df = pd.read_csv('dataset/classification/aggregated_results_by_ds__pool_28797724__2021_10_19.tsv', sep='\t')
labelled_df = labelled_df.drop('Unnamed: 3', axis=1)
labelled_df['INPUT:images'] = labelled_df['INPUT:images'].apply(lambda x: 'dataset' + x[13:])
labelled_df['OUTPUT:result'] = (labelled_df['OUTPUT:result'] == 'ok').astype(int)
labelled_df['CONFIDENCE:result'] = labelled_df['CONFIDENCE:result'].apply(lambda x: float(x[:-1])/100)

In [6]:
labelled_df = labelled_df[labelled_df['CONFIDENCE:result'] > 0.95]
print("# Confident labels: ", len(labelled_df))
labelled_df = labelled_df.drop('CONFIDENCE:result', axis=1)
labelled_df.columns = ['img_path', 'label']
labelled_df.head()

# Confident labels:  8452


,img_path,label
0,dataset/classification/train_unlabelled/6551.png,0
1,dataset/classification/train_unlabelled/2723.png,1
2,dataset/classification/train_unlabelled/3777.png,0
3,dataset/classification/train_unlabelled/2858.png,1
4,dataset/classification/train_unlabelled/543.png,1


In [7]:
train_df = pd.read_csv('dataset/classification/train_labelled.csv')
train_df.head()

,img_path,label
0,dataset/classification/train_labelled/0.png,0
1,dataset/classification/train_labelled/1.png,0
2,dataset/classification/train_labelled/2.png,0
3,dataset/classification/train_labelled/3.png,0
4,dataset/classification/train_labelled/4.png,0


In [8]:
train_df = train_df.append(labelled_df, ignore_index=True)

In [9]:
train_image_paths = train_df.img_path.values
train_labels = train_df.label.values

In [10]:
len(train_image_paths)

9452

In [11]:
ipyplot.plot_class_tabs(train_image_paths, train_labels)

# Split dataset

In [12]:
total_len = len(train_image_paths)
n_train = int(0.7 * total_len)
n_valid = int(0.2 * total_len)
n_test = total_len - n_train - n_valid

In [13]:
import numpy as np
np.random.seed(420)

In [14]:
rand_idcs = np.random.permutation(train_df.index)
train_idcs = rand_idcs[:n_train]
val_idcs = rand_idcs[n_train:n_train+n_valid]
test_idcs = rand_idcs[n_train+n_valid:n_train+n_valid+n_test]

In [15]:
train_X, train_y = train_df.loc[train_idcs,'img_path'], train_df.loc[train_idcs, 'label']
val_X, val_y = train_df.loc[val_idcs,'img_path'], train_df.loc[val_idcs, 'label']
test_X, test_y = train_df.loc[test_idcs,'img_path'], train_df.loc[test_idcs, 'label']

# Prepare model

In [16]:
train_dataset = ImagesDataset(list(train_X), list(train_y), transform=transform)
val_dataset = ImagesDataset(list(val_X), list(val_y), transform=transform)
test_dataset = ImagesDataset(list(test_X), list(test_y), transform=transform)

In [17]:
from matplotlib import pyplot as plt

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=params.BATCH_SIZE,
                                           num_workers=16,
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=params.BATCH_SIZE,
                                           num_workers=16,
                                           shuffle=False)

In [19]:
weights = torch.Tensor([10,1])#torch.Tensor(np.bincount(train_y).tolist())

In [20]:
# Run macos_cert_fix.py if you are getting ssl errors
model = ImageClassifier(class_weight=weights, learning_rate=params.LEARNING_RATE)

In [33]:
model = ImageClassifier(trunk=model, class_weight=weights, learning_rate=1e-4)

In [34]:
trainer = pl.Trainer(
    max_epochs=5, #params.MAX_EPOCHS,
    log_every_n_steps=1, # Change to enable tensorboard logging
#     accelerator="dp",
    gpus=[1] # Uncomment to use GPU training
)
trainer.fit(model, train_loader, val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name      | Type             | Params
-----------------------------------------------
0 | trunk     | ImageClassifier  | 2.2 M 
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | valid_acc | Accuracy         | 0     
-----------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.906     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Compute training set accuracy

In [26]:
from sklearn.metrics import accuracy_score, classification_report

In [40]:
labels, predictions = evaluate_model(model, test_dataset, num_workers=20, device='cuda:0')

  0%|          | 0/30 [00:01<?, ?it/s]

In [36]:
f1_score(labels, predictions, labels=1, average='binary')

0.922077922077922

In [37]:
accuracy_score(labels, predictions)

0.9873015873015873

In [38]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1730
           1       0.96      0.89      0.92       160

    accuracy                           0.99      1890
   macro avg       0.97      0.94      0.96      1890
weighted avg       0.99      0.99      0.99      1890



In [46]:
from torchvision import transforms
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize((50, 200)), # Hint: this might not be the best way to resize images
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Hint: this might not be the best normalization
     ]
)
test_dataset = ImagesDataset(list(test_X), list(test_y), transform=test_transform)

In [47]:
labels, predictions = evaluate_model(model, test_dataset, num_workers=20, device='cuda:0')

  0%|          | 0/30 [00:01<?, ?it/s]

In [48]:
f1_score(labels, predictions, labels=1, average='binary')

0.9689440993788819

In [49]:
accuracy_score(labels, predictions)

0.9947145877378436

In [50]:
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       863
           1       1.00      0.94      0.97        83

    accuracy                           0.99       946
   macro avg       1.00      0.97      0.98       946
weighted avg       0.99      0.99      0.99       946



# Save model

In [61]:
trainer.save_checkpoint("model.ckpt")

# Inference

In [60]:
from pathlib import Path
inf_imgs = list(Path("dataset/classification/test/").glob("*.png"))
inf_dataset = ImagesDataset(inf_imgs, transform=test_transform)

In [63]:
_, predictions = evaluate_model(model, inf_dataset, batch_size=32, num_workers=20, device='cuda:0')

  0%|          | 0/184 [00:01<?, ?it/s]

In [65]:
# Only the image name will be used when evaluating, the rest of the path does not matter
submission_df = pd.DataFrame({'img_path': inf_imgs, 'label': predictions})


In [70]:
submission_df['img_path'] = submission_df['img_path'].apply(lambda x: x.name)

In [72]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!git clone 